# R in Snowflake Workspace Notebooks

This notebook demonstrates how to use R within Snowflake Workspace Notebooks using rpy2.

**Capabilities:**
- Execute R code in `%%R` magic cells alongside Python
- Transfer data bidirectionally between Python and R
- Connect to Snowflake from R using ADBC (with PAT authentication)

**Sections:**
1. [Installation & Configuration](#section-1-installation--configuration)
2. [Python & R Interoperability](#section-2-python--r-interoperability)
3. [Snowflake Database Connectivity](#section-3-snowflake-database-connectivity)

---

# Section 1: Installation & Configuration

This section sets up R and rpy2 in the Workspace Notebook environment.

## Overview

Snowflake Workspace Notebooks run in containers with a managed Python kernel. To use R:

1. **Install R** via micromamba (lightweight conda-compatible package manager)
2. **Install rpy2** into the notebook's Python kernel
3. **Register `%%R` magic** for R cell support

## Customizing R Packages

Edit `r_packages.yaml` to customize which R packages are installed:

```yaml
# Conda-forge packages (installed via micromamba)
conda_packages:
  - r-base           # Required: Base R
  - r-tidyverse      # Data manipulation
  - r-yourpackage    # Add packages here

# CRAN packages (installed via install.packages)
cran_packages:
  - somepackage      # Packages not available on conda-forge
```

## Installation Options

| Command | Description |
|---------|-------------|
| `bash setup_r_environment.sh` | Basic R installation |
| `bash setup_r_environment.sh --adbc` | R + ADBC driver for Snowflake connectivity |
| `bash setup_r_environment.sh --verbose` | Show detailed logging |
| `bash setup_r_environment.sh --help` | Show all options |

### 1.1 Install R Environment

Run the setup script. Choose `--basic` for R only, or `--adbc` to include ADBC Snowflake driver.

**Note:** This step takes 2-5 minutes on first run. The `--adbc` option takes longer as it compiles the Snowflake driver.

The script includes:
- Pre-flight checks (disk space, network connectivity)
- Automatic retry for network operations
- Logging to `setup_r.log` for debugging

In [ ]:
# Choose ONE of the following:

# Option A: Basic R installation (faster)
# !bash setup_r_environment.sh --basic

# Option B: R + ADBC for Snowflake connectivity (required for Section 3)
!bash setup_r_environment.sh --adbc

### 1.2 Configure Python Environment & Install rpy2

This cell uses the helper module to:
1. Point Python to the R environment
2. Install rpy2 into the notebook kernel
3. Register the `%%R` magic

**Run this cell after the installation script completes.**

In [ ]:
# Method 1: Using the helper module (recommended)
import sys
sys.path.insert(0, '.')  # Ensure current directory is in path

from r_helpers import setup_r_environment

result = setup_r_environment()

if result['success']:
    print("✓ R environment configured successfully")
    print(f"  R version: {result['r_version']}")
    print(f"  rpy2 installed: {result['rpy2_installed']}")
    print(f"  %%R magic registered: {result['magic_registered']}")
else:
    print("✗ Setup failed:")
    for error in result['errors']:
        print(f"  - {error}")

In [ ]:
# Method 2: Manual configuration (if helper module not available)
# Uncomment and run if Method 1 fails

# import os
# import sys
# import subprocess

# ENV_PREFIX = "/root/.local/share/mamba/envs/r_env"
# os.environ["PATH"] = f"{ENV_PREFIX}/bin:" + os.environ["PATH"]
# os.environ["R_HOME"] = f"{ENV_PREFIX}/lib/R"

# subprocess.run([sys.executable, "-m", "pip", "install", "rpy2", "-q"], check=True)

# from rpy2.ipython import rmagic
# get_ipython().register_magics(rmagic.RMagics)
# print("R environment configured")

### 1.3 Verify R Installation

Test that R is working correctly.

In [ ]:
%%R
# Print R version
R.version.string

In [ ]:
%%R
# List installed packages
ip <- as.data.frame(installed.packages()[, c(1, 3:4)])
ip <- ip[is.na(ip$Priority), 1:2, drop = FALSE]
print(ip)

### 1.4 Run Diagnostics (Optional)

Run comprehensive environment diagnostics to verify all components are working.

In [ ]:
from r_helpers import check_environment, print_diagnostics

# Run and display diagnostics
print_diagnostics()

---

# Section 2: Python & R Interoperability

This section demonstrates how to work with data in both Python and R, including:
- Using the `%%R` magic for R cells
- Passing data from Python to R
- Passing data from R to Python
- Running R functions from Python

## 2.1 Using %%R Magic Cells

The `%%R` magic lets you write R code directly in a cell. The magic supports flags:

| Flag | Description |
|------|-------------|
| `-i var` | Import Python variable `var` into R |
| `-o var` | Export R variable `var` back to Python |
| `-w WIDTH` | Set plot width |
| `-h HEIGHT` | Set plot height |

In [ ]:
%%R
# Basic R operations
x <- c(1, 2, 3, 4, 5)
mean(x)

In [ ]:
%%R
# Using tidyverse
library(dplyr)

data.frame(
  name = c("Alice", "Bob", "Charlie"),
  score = c(85, 92, 78)
) %>%
  mutate(grade = case_when(
    score >= 90 ~ "A",
    score >= 80 ~ "B",
    TRUE ~ "C"
  ))

## 2.2 Passing Data: Python → R

Use the `-i` flag to pass Python objects into R cells.

In [ ]:
# Create a pandas DataFrame in Python
import pandas as pd

python_df = pd.DataFrame({
    'city': ['New York', 'Los Angeles', 'Chicago', 'Houston'],
    'population': [8336817, 3979576, 2693976, 2320268],
    'area_sq_mi': [302.6, 468.7, 227.3, 670.6]
})

print("Python DataFrame:")
python_df

In [ ]:
%%R -i python_df
# The Python DataFrame is now available in R as 'python_df'
library(dplyr)

cat("Received DataFrame in R:\n")
glimpse(python_df)

# Perform R operations
python_df %>%
  mutate(density = population / area_sq_mi) %>%
  arrange(desc(density))

## 2.3 Passing Data: R → Python

Use the `-o` flag to export R objects back to Python.

In [ ]:
%%R -o r_result
# Create a data frame in R
r_result <- data.frame(
  x = 1:10,
  y = (1:10)^2,
  label = paste0("Point_", 1:10)
)

cat("Created R data.frame:\n")
print(r_result)

In [ ]:
# The R data.frame is now available in Python
print("R result in Python:")
print(type(r_result))
print(r_result)

## 2.4 Using R from Python (without magic)

For more control, you can use rpy2's Python API directly.

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# Import R packages
base = importr('base')
stats = importr('stats')

# Run R code and get results
result = ro.r('sum(1:100)')
print(f"Sum of 1 to 100: {result[0]}")

In [ ]:
# Convert pandas DataFrame to R and run R functions on it
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

# Create sample data
df = pd.DataFrame({
    'x': [1, 2, 3, 4, 5],
    'y': [2.1, 3.9, 6.2, 7.8, 10.1]
})

# Convert to R and run linear regression
with (ro.default_converter + pandas2ri.converter).context():
    r_df = ro.conversion.get_conversion().py2rpy(df)

# Run linear regression in R
lm_result = stats.lm('y ~ x', data=r_df)
print("Linear Regression Results:")
print(base.summary(lm_result))

## 2.5 Working with R's Built-in Datasets

Access R's built-in datasets and convert them to pandas.

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# Load the iris dataset in R
ro.r("data(iris)")

# Get the R data.frame
iris_r = ro.r["iris"]

# Convert to pandas DataFrame
with localconverter(ro.default_converter + pandas2ri.converter):
    iris_df = pandas2ri.rpy2py(iris_r)

print("Iris dataset (first 10 rows):")
iris_df.head(10)

---

# Section 3: Snowflake Database Connectivity

This section demonstrates connecting to Snowflake from R using ADBC.

**Prerequisites:**
- Run the setup script with `--adbc` flag (Section 1.1)
- Have appropriate Snowflake permissions

## Authentication Options

| Method | Status | Notes |
|--------|--------|-------|
| Python `get_active_session()` | ✅ Works | Use for Snowpark queries, bridge to R via rpy2 |
| ADBC with PAT | ✅ Works | Direct R-to-Snowflake, requires PAT token |
| SPCS OAuth Token | ❌ Blocked | Container token not authorized for ADBC |
| Username/Password | ❌ Blocked | SPCS requires OAuth |

## Connection Management

This notebook uses connection pooling - the ADBC connection is stored as `r_sf_con` in R's global environment and reused across cells. This avoids the overhead of creating new connections for each query.

## 3.1 Setup Python Session

First, establish the standard Python Snowpark session.

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Print connection details
print("Snowflake Connection:")
print(f"  Account:   {session.sql('SELECT CURRENT_ACCOUNT()').collect()[0][0]}")
print(f"  User:      {session.sql('SELECT CURRENT_USER()').collect()[0][0]}")
print(f"  Role:      {session.get_current_role()}")
print(f"  Database:  {session.get_current_database()}")
print(f"  Schema:    {session.get_current_schema()}")
print(f"  Warehouse: {session.get_current_warehouse()}")

## 3.2 Create Programmatic Access Token (PAT)

ADBC requires a PAT for authentication. Use the `PATManager` helper class for token management.

**Features:**
- Automatic expiry tracking
- Token refresh when needed
- Secure environment variable storage

**Security Notes:**
- PAT is stored in environment variable (not persisted)
- Token expires after specified duration
- Use `ROLE_RESTRICTION` to limit token scope

In [ ]:
from r_helpers import PATManager

# Initialize PAT manager
pat_mgr = PATManager(session)

# Create a new PAT (or refresh existing one)
result = pat_mgr.create_pat(
    days_to_expiry=1,           # Token valid for 1 day
    force_recreate=True,        # Replace existing token
    network_policy_bypass_mins=240  # 4 hour bypass
)

if result['success']:
    print(f"✓ PAT created successfully")
    print(f"  User: {result['user']}")
    print(f"  Role restriction: {result['role_restriction']}")
    print(f"  Expires: {result['expires_at']}")
else:
    print(f"✗ PAT creation failed: {result['error']}")

In [ ]:
# Check PAT status at any time
status = pat_mgr.get_status()
print("PAT Status:")
for key, value in status.items():
    print(f"  {key}: {value}")

## 3.3 Validate ADBC Prerequisites

Before connecting, validate that all ADBC prerequisites are met.

In [ ]:
from r_helpers import validate_adbc_connection

valid, message = validate_adbc_connection()
print(message)

## 3.4 Initialize R Connection Management

Load the connection management functions into R. This provides:
- `get_snowflake_connection()` - Get or create connection (stored as `r_sf_con`)
- `close_snowflake_connection()` - Close and release connection
- `is_snowflake_connected()` - Check connection status
- `snowflake_connection_status()` - Get detailed status

In [ ]:
from r_helpers import init_r_connection_management

success, msg = init_r_connection_management()
print(msg)

## 3.5 Connect to Snowflake from R (ADBC)

Use `get_snowflake_connection()` to establish or reuse the ADBC connection.

The connection is stored as `r_sf_con` in R's global environment and is automatically reused in subsequent cells.

In [ ]:
%%R
# Get or create the Snowflake connection
# Connection is stored globally as r_sf_con
r_sf_con <- get_snowflake_connection()

# Show connection status
cat("\nConnection Status:\n")
status <- snowflake_connection_status()
cat("  Connected:", status$connected, "\n")
cat("  Account:  ", status$account, "\n")
cat("  User:     ", status$user, "\n")
cat("  Database: ", status$database, "\n")

## 3.6 Query Snowflake from R

Run queries using the `r_sf_con` connection. The connection is automatically reused across cells.

In [ ]:
%%R
# Simple test query using r_sf_con
r_sf_con |>
  read_adbc("SELECT CURRENT_USER() AS USER, CURRENT_ROLE() AS ROLE, CURRENT_WAREHOUSE() AS WAREHOUSE") |>
  tibble::as_tibble()

In [ ]:
%%R
# Query sample data from Snowflake
# Using the shared SNOWFLAKE_SAMPLE_DATA database
nations <- r_sf_con |>
  read_adbc("
    SELECT N_NATIONKEY, N_NAME, N_REGIONKEY 
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION 
    ORDER BY N_NATIONKEY
    LIMIT 10
  ") |>
  tibble::as_tibble()

print(nations)

In [ ]:
%%R
# More complex query with aggregation
library(dplyr)

orders_summary <- r_sf_con |>
  read_adbc("
    SELECT 
      O_ORDERSTATUS,
      COUNT(*) as ORDER_COUNT,
      SUM(O_TOTALPRICE) as TOTAL_VALUE,
      AVG(O_TOTALPRICE) as AVG_VALUE
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
    GROUP BY O_ORDERSTATUS
    ORDER BY ORDER_COUNT DESC
  ") |>
  tibble::as_tibble()

print(orders_summary)

In [ ]:
%%R
# Verify connection is being reused (not recreated)
cat("Connection still valid:", is_snowflake_connected(), "\n")

## 3.7 Query from Python, Analyze in R

An alternative pattern: use Python's Snowpark session for querying, then pass data to R for analysis.

In [ ]:
# Query using Python Snowpark session
customers_df = session.sql("""
    SELECT 
        C_CUSTKEY,
        C_NAME,
        C_NATIONKEY,
        C_ACCTBAL
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
    LIMIT 100
""").to_pandas()

print(f"Retrieved {len(customers_df)} rows")
customers_df.head()

In [ ]:
%%R -i customers_df
# Analyze the data in R
library(dplyr)

cat("Summary Statistics for Customer Account Balance:\n")
summary(customers_df$C_ACCTBAL)

cat("\nCustomers by Nation (top 5):\n")
customers_df %>%
  group_by(C_NATIONKEY) %>%
  summarise(
    count = n(),
    avg_balance = mean(C_ACCTBAL),
    total_balance = sum(C_ACCTBAL)
  ) %>%
  arrange(desc(count)) %>%
  head(5)

## 3.8 Check Connection Status

You can check the connection status from either Python or R.

In [ ]:
# Check status from Python
from r_helpers import get_r_connection_status

status = get_r_connection_status()
print("R Connection Status (from Python):")
for key, value in status.items():
    print(f"  {key}: {value}")

In [ ]:
%%R
# Check status from R
cat("R Connection Status (from R):\n")
status <- snowflake_connection_status()
cat("  connected: ", status$connected, "\n")
cat("  con_exists:", status$con_exists, "\n")
cat("  db_exists: ", status$db_exists, "\n")
cat("  pat_set:   ", status$pat_set, "\n")

## 3.9 Clean Up

Close ADBC connection and optionally remove the PAT.

In [ ]:
%%R
# Close the Snowflake connection
close_snowflake_connection()

In [ ]:
# Alternative: Close from Python
# from r_helpers import close_r_connection
# success, msg = close_r_connection()
# print(msg)

In [ ]:
# Remove PAT when done (optional)
# pat_mgr.remove_pat()
# print("PAT removed")

---

## Troubleshooting

### Common Issues

| Issue | Solution |
|-------|----------|
| `ModuleNotFoundError: No module named 'rpy2'` | Run Section 1.2 to install rpy2 |
| `R.version.string` returns error | Verify PATH and R_HOME are set correctly |
| ADBC `auth_pat` error | Ensure PAT was created and stored in `SNOWFLAKE_PAT` |
| Network policy error | PAT may need `MINS_TO_BYPASS_NETWORK_POLICY_REQUIREMENT` |
| `adbcsnowflake` not found | Ensure setup script ran with `--adbc` flag |
| Setup script fails | Check `setup_r.log` for detailed error messages |
| `r_sf_con` not found | Run `get_snowflake_connection()` to create connection |

### Run Full Diagnostics

In [ ]:
# Comprehensive diagnostic check
from r_helpers import print_diagnostics
print_diagnostics()

In [ ]:
# Quick environment check
import os
import shutil

print("Quick Environment Check:")
print(f"  R_HOME: {os.environ.get('R_HOME', 'NOT SET')}")
print(f"  R binary: {shutil.which('R') or 'NOT FOUND'}")
print(f"  SNOWFLAKE_ACCOUNT: {os.environ.get('SNOWFLAKE_ACCOUNT', 'NOT SET')}")
print(f"  SNOWFLAKE_PAT: {'SET' if os.environ.get('SNOWFLAKE_PAT') else 'NOT SET'}")

In [ ]:
# View setup log if something went wrong
# !tail -50 setup_r.log